# Mummer pipeline 

**Versions**  
Mummer: 4.0.0beta2  
Python: 3.7.3  
GNUPLOT: 4.6.0  
ghostscript: 9.22  
Mercator: 2013.01.11

### Mercator

Run the following perl script to identify blocks of synteny:  
Run_mercator.pl

In [ ]:
#!/usr/bin/perl -w

use strict;

my $ref = shift;
my $query = shift;

$ref =~ m/(.+)\.fa/;
my $refBase = $1;
$query =~ m/(.+)\.fa/;
my $queryBase = $1;

#Repeatmasker
# Mask low complexity repeats
my $refLow = $ref.".low";
my $refLowMasked = $refLow.".masked";

my $queryLow = $query.".low";
my $queryLowMasked = $queryLow.".masked";

system("ln -s $ref $refLow");
system("ln -s $query $queryLow");
system("RepeatMasker -e ncbi -pa 28 -q -no_is -noint -species drosophila $refLow >& RM.refLOW.log");
system("RepeatMasker -e ncbi -pa 28 -q -no_is -noint -species drosophila $queryLow >& RM.queryLOW.log");

# Mask interspersed repeats
my $refInt = $ref.".int";
my $refIntMasked = $refInt.".masked";

my $queryInt = $query.".int";
my $queryIntMasked = $queryInt.".masked";

system("ln -s $ref $refInt");
system("ln -s $query $queryInt");
system("RepeatMasker -e ncbi -pa 28 -q -no_is -nolow -species drosophila $refInt >& RM.refINT.log");
system("RepeatMasker -e ncbi -pa 28 -q -no_is -nolow -species drosophila $queryInt >& RM.queryINT.log");

die unless -s $refLowMasked;
die unless -s $queryLowMasked;
die unless -s $refIntMasked;
die unless -s $queryIntMasked;

# Merge masking into one hardmasked file
my $refMasked = $ref.".masked";
my $queryMasked = $query.".masked";

print "nmerge $refIntMasked $refLowMasked > $refMasked\n";
print "nmerge $queryIntMasked $queryLowMasked > $queryMasked\n";
system("nmerge $refIntMasked $refLowMasked > $refMasked");
system("nmerge $queryIntMasked $queryLowMasked > $queryMasked");

die unless -s $refMasked;
die unless -s $queryMasked;

# Create softmasked file
my $refSoftMasked = $ref.".softmasked";
my $querySoftMasked = $query.".softmasked";
system("faSoftMask $ref $refMasked > $refSoftMasked");
system("faSoftMask $query $queryMasked > $querySoftMasked");

die unless -s $refSoftMasked;
die unless -s $querySoftMasked;

#Create SDB files
my $refSDB = $refBase.".sdb";
my $querySDB = $queryBase.".sdb";
system("cat $refSoftMasked | fa2sdb -c $refSDB");
system("cat $querySoftMasked | fa2sdb -c $querySDB");

die unless -s $refSDB;
die unless -s $querySDB;

# Run SNAP with fly HMM
my $refZFF = $refBase.".zff";
my $refGFF = $refBase.".gff";
my $queryZFF = $queryBase.".zff";
my $queryGFF = $queryBase.".gff";
system("runSnap /home/cee53/pkg/snap/HMM/fly < $refIntMasked > $refZFF");
system("runSnap /home/cee53/pkg/snap/HMM/fly < $queryIntMasked > $queryZFF");

die unless -s $refZFF;
die unless -s $queryZFF;

system("cat $refZFF | zff2gtf --source=SNAP > $refGFF");
system("cat $queryZFF | zff2gtf --source=SNAP > $queryGFF");

die unless -s $refGFF;
die unless -s $queryGFF;

my $tree = "($refBase\:0.05,$queryBase\:0.05)\;";

# Create input files for Mercator
system("makeMercatorInput $refBase $queryBase");

# Make treefile
system("echo \'$tree\' > treefile");

#Move files into input dir
system("mkdir input");
system("mkdir output");
system("mv $refBase\* input/.");
system("mv $queryBase\* input/.");
system("mv treefile input/.");

#Constructing Orthology Map
system("mercator -i input -o output $refBase $queryBase");

#Move to output dir
chdir("output");
system("ln -s ../input/*.sdb .");
system("ln -s ../input/treefile");

#Breakpoint Finding
# Convert the orthology map into a more general homology map
system("omap2hmap genomes < pre.map > pre.h.map");
# Create the graph relating the breakpoint regions
system("makeBreakpointGraph pre.h.map treefile");
# Make pairwise alignments for breakpoint regions
system("mkdir bp_alignments");
system("makeBreakpointAlignmentInput --out-dir=bp_alignments");
system("mavidAlignDirs --init-dir=bp_alignments");
# Find a good configuration of breakpoints
system("findBreakpoints --resolution 1000 pre.h.map treefile edges bp_alignments > breakpoints");
# Refine the map by splitting the breakpoint regions
system("breakMap breakpoints < pre.h.map > better.h.map");
# Convert back to the orthology map format
system("hmap2omap genomes < better.h.map > better.map");

#Generate MAVID input
#Convert pairwise hits to alignment constraints
system("phits2constraints -i ../input < pairwisehits > constraints");
#Create directories and files for alignment
system("mkdir alignments");
system("makeAlignmentInput --map=better.map . alignments");

#Align with MAVID
#Align all sequence files in directory structure
system("mavidAlignDirs --init-dir=alignments");

Determine which _D. triauraria_ megascaffold corresponds to which Muller element based on chromosome count table from <tt>Genome_Assembly<tt> pipeline. 

**_D. triauraria_**  

|Muller element | Scaffold |
|--- | ---
Muller_ A|HiC_Scaffold_5|
Muller_B |HiC_scaffold_1|
Muller_C |HiC_scaffold_2|
Muller_D |HiC_scaffold_7|
Muller_E |HiC_scaffold_8|
Muller_F |HiC_scaffold_6|

Run the python script called <tt> extract_scaff_rename_muller.py <tt>  

#### Python script to extract scaffolds and rename as Muller element. Can reverse complement scaffolds as well if necessary.

instructions for "extract_scaff_rename_muller.py"
Script is to rename HiC scaffolds from Drosophila species to appropriate Muller elements and reverse complement the scaffold if necessary.

Be sure to input two output files, the first will print the Muller elements in the order they are found and the second will be the reordered Muller elements (A,B,C,D,E,F).

in bash shell type:
python extract_scaff_rename_muller.py -h 

Output will be 
Options:
  -h, --help            show this help message and exit
  -i FASTA              scaffolds.fasta
  -o OUTFILE            output1.fasta
  -t OUTFILE2, --o2=OUTFILE2
                        output2.fasta - this will be the correctly ordered
                        file
  -a MA                 scaffold corresp. to Muller A
  -b MB                 scaffold corresp. to Muller B
  -c MC                 scaffold corresp. to Muller C
  -d MD                 scaffold corresp. to Muller D
  -e ME                 scaffold corresp. to Muller E
  -f MF                 scaffold corresp. to Muller F
  -u MAR, --rcMA=MAR    scaffold corresp. to Muller A - use to reverse
                        complement scaff
  -v MBR, --rcMB=MBR    scaffold corresp. to Muller B - use to reverse
                        complement scaff
  -w MCR, --rcMC=MCR    scaffold corresp. to Muller C - use to reverse
                        complement scaff
  -x MDR, --rcMD=MDR    scaffold corresp. to Muller D - use to reverse
                        complement scaff
  -y MER, --rcME=MER    scaffold corresp. to Muller E - use to reverse
                        complement scaff
  -z MFR, --rcMF=MFR    scaffold corresp. to Muller F - use to reverse
                        complement scaff


After running the script, input OUTFILE2 name into the <OUTFILE2> slot in the nucmer_slurm.sh script in the folder. Update prefix for nucmer output files, the folder for the SLURM output, and the nucmer parameters as necessary.  


In [ ]:
#/projects/genetics/ellison_lab/nicole/mummer_projects/script/
from optparse import OptionParser
from Bio import SeqIO

parser = OptionParser()
parser.add_option("-i", dest="fasta", type="string", action="store", help = "scaffolds.fasta")
parser.add_option("-o", dest="outfile", type = "string", action = "store", help = "output1.fasta") 
parser.add_option("-t","--o2", dest="outfile2", type = "string", action = "store", help = "output2.fasta - this will be the correctly ordered file")
parser.add_option("-a", dest="MA", type = "string", help = "scaffold corresp. to Muller A")
parser.add_option("-b", dest="MB", type = "string", help = "scaffold corresp. to Muller B")
parser.add_option("-c", dest="MC", type = "string", help = "scaffold corresp. to Muller C")
parser.add_option("-d", dest="MD", type = "string", help = "scaffold corresp. to Muller D")
parser.add_option("-e", dest="ME", type = "string", help = "scaffold corresp. to Muller E")
parser.add_option("-f", dest="MF", type = "string", action = "store", help = "scaffold corresp. to Muller F")
parser.add_option("-u","--rcMA", dest="MAR", type = "string", action = "store", help = "scaffold corresp. to Muller A - use to reverse complement scaff")
parser.add_option("-v", "--rcMB", dest="MBR", type = "string", action = "store", help = "scaffold corresp. to Muller B - use to reverse complement scaff")
parser.add_option("-w", "--rcMC", dest="MCR", type = "string", action = "store", help = "scaffold corresp. to Muller C - use to reverse complement scaff")
parser.add_option("-x", "--rcMD", dest="MDR", type = "string", action = "store", help = "scaffold corresp. to Muller D - use to reverse complement scaff")
parser.add_option("-y", "--rcME", dest="MER", type = "string", action = "store", help = "scaffold corresp. to Muller E - use to reverse complement scaff")
parser.add_option("-z", "--rcMF", dest="MFR", type = "string", action = "store", help = "scaffold corresp. to Muller F - use to reverse complement scaff")

(options, args) = parser.parse_args()
f = open(options.outfile, 'w')

for record in SeqIO.parse(options.fasta, "fasta"):
	if record.id == options.MA:
		MA = record	
		f.write(">" + "Muller_A" + '\n' + str(MA.seq) + '\n')
	elif record.id == options.MB:
		MB = record
		f.write(">" + "Muller_B" + '\n' + str(MB.seq) + '\n')
	elif record.id == options.MC:
		MC = record
		f.write(">" + "Muller_C" + '\n' + str(MC.seq) + '\n')
	elif record.id == options.MD:
		MD = record
		f.write(">" + "Muller_D" + '\n' + str(MD.seq) + '\n')
	elif record.id == options.ME:
		ME = record
		f.write(">" + "Muller_E" + '\n' + str(ME.seq) + '\n')
	elif record.id == options.MF:
		MF = record
		f.write(">" + "Muller_F" + '\n' + str(MF.seq) + '\n')

for record in SeqIO.parse(options.fasta, "fasta"):
        if record.id == options.MAR:
                MAR = record
                f.write(">" + "Muller_A" + '\n' + str(MAR.seq.reverse_complement()) + '\n')
        elif record.id == options.MBR:
                MBR = record
                f.write(">" + "Muller_B" + '\n' + str(MBR.seq.reverse_complement()) + '\n')
        elif record.id == options.MCR:
                MCR = record
                f.write(">" + "Muller_C" + '\n' + str(MCR.seq.reverse_complement()) + '\n')
        elif record.id == options.MDR:
                MDR = record
                f.write(">" + "Muller_D" + '\n' + str(MDR.seq.reverse_complement()) + '\n')
        elif record.id == options.MER:
                MER = record
                f.write(">" + "Muller_E" + '\n' + str(MER.seq.reverse_complement()) + '\n')
        elif record.id == options.MFR:
                MFR = record
                f.write(">" + "Muller_F" + '\n' + str(MFR.seq.reverse_complement()) + '\n')

f.close()

r = open(options.outfile2, 'w')

for record in SeqIO.parse(options.outfile, "fasta"):
	if record.id == "Muller_A":
		A = record
	if record.id == "Muller_B":
		B = record
	if record.id == "Muller_C":
                C = record
	if record.id == "Muller_D":
		D = record
	if record.id == "Muller_E":
                E = record
	if record.id == "Muller_F":
                F = record



r.write(">" + str(A.id) + '\n' + str(A.seq) + '\n')
r.write(">" + str(B.id) + '\n' + str(B.seq) + '\n')
r.write(">" + str(C.id) + '\n' + str(C.seq) + '\n')   
r.write(">" + str(D.id) + '\n' + str(D.seq) + '\n')    
r.write(">" + str(E.id) + '\n' + str(E.seq) + '\n')
r.write(">" + str(F.id) + '\n' + str(F.seq) + '\n')
r.close()

### MUMmer
Create dotplot comparison between _D.triauraria_ and _D. melanogaster_ genomes.

In [ ]:
#!/bin/bash

#SBATCH --partition=main    # Partition (job queue)
#SBATCH --job-name=mummer         # Assign an 8-character name to your job, no spaces
#SBATCH --nodes=1                # Number of compute nodes
#SBATCH --ntasks=1               # Processes (usually = cores) on each node
#SBATCH --cpus-per-task=28       # Threads per process (or per core)
#SBATCH --export=ALL             # Export you current environment settings to the job environment
#SBATCH --time=12:00:00
#SBATCH --mem=20G
#SBATCH --output=/scratch/nt365/mummer/mummer_projects/dmel_dtria/slurm-%A_%a.out

source activate MUMMER
promer --mum -b 10 -l 50 --coords -p promer_mum_dtria_rcMAMB dmel-all-chromosome-r6.22_extract_ABCDEF.fasta dtria_muller_o.fasta

mummerplot -p promer_mum_dtria_rcMAMB -t postscript promer_mum_dtria_rcMAMB.delta

export GNUPLOT_PS_DIR=/home/nt365/pkg/gnuplot-4.6.0/term/PostScript 
gnuplot promer_mum_dtria_rcMAMB.gp
ps2pdf promer_mum_dtria_rcMAMB.ps